In [4]:
import cv2
import numpy as np
import os
import mediapipe as mp

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from scipy import stats
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from jproperties import Properties

In [5]:
configs = Properties()
with open('../.properties', 'rb') as config_file: configs.load(config_file)
CAMERA_INDEX = int(configs.get('CAMERA_INDEX').data)

In [6]:
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

In [7]:
DATA_PATH = os.path.join('dist') 
num_videos = 20
sequence_length = 120

if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
# clear dist
for file in os.listdir(DATA_PATH):
    os.remove(os.path.join(DATA_PATH, file))

# Registraiamo video

In [7]:
cap = cv2.VideoCapture(CAMERA_INDEX)

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

for idx_video in range(num_videos):

    video_filename = os.path.join(DATA_PATH, f"{idx_video}.mp4")
    writer= cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))

    for idx_frame in range(sequence_length):
        ret, frame = cap.read()

        if idx_frame == 0:
            cv2.putText(frame, f'Stating Video #{idx_video+1}', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', frame)
            cv2.waitKey(3000)
        else:
            cv2.imshow('Video', frame)

        writer.write(frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    writer.release()
                
cap.release()
cv2.destroyAllWindows()

2023-10-11 18:20:43.920 python[35202:6085660] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 

## Mostriamo e basta

In [ ]:
cap = cv2.VideoCapture(CAMERA_INDEX)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Converti il frame in scala di grigi
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Esegui il rilevamento delle pose con MediaPipe
    results = holistic.process(frame_rgb)

    # Estrai le coordinate della spalla destra, del gomito destro e della mano destra
    if results.pose_landmarks:
        right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
        right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]

        # Le coordinate sono normalizzate, quindi convertile in coordinate pixel
        height, width, _ = frame.shape
        shoulder_x, shoulder_y = int(right_shoulder.x * width), int(right_shoulder.y * height)
        elbow_x, elbow_y = int(right_elbow.x * width), int(right_elbow.y * height)
        wrist_x, wrist_y = int(right_wrist.x * width), int(right_wrist.y * height)

        # Disegna i punti sul frame
        cv2.circle(frame, (shoulder_x, shoulder_y), 5, (0, 0, 255), -1)
        cv2.circle(frame, (elbow_x, elbow_y), 5, (0, 255, 0), -1)
        cv2.circle(frame, (wrist_x, wrist_y), 5, (255, 0, 0), -1)

    # Visualizza il frame con le coordinate
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Modello

In [ ]:
window_len = 10
X_training = []
y_training = []
frame_buffer = []
video_paths = [os.path.join(DATA_PATH, f"{idx_video}.mp4") for idx_video in range(num_videos)]

In [8]:
for idx_video, video_path in enumerate(video_paths):
    cap = cv2.VideoCapture(video_path)
    print(f"Processing video #{idx_video+1}...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

        if results.pose_landmarks:
            #right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            #right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
            right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]
            
            # Estrai le coordinate x, y, e z e normalizzale
            #shoulder_coords = [right_shoulder.x, right_shoulder.y, right_shoulder.z]
            #elbow_coords = [right_elbow.x, right_elbow.y, right_elbow.z]
            wrist_coords = [
                right_wrist.x, 
                right_wrist.y, 
                #right_wrist.z
                ]

            frame_buffer.append(
                # shoulder_coords
                #  + elbow_coords 
                wrist_coords
            )

            # Se il buffer contiene abbastanza frame, aggiungilo a X_training e y_training
            if len(frame_buffer) >= window_len:
                X_training.append(frame_buffer[:int(window_len/2)])
                y_training.append(frame_buffer[-1])
                frame_buffer = frame_buffer[1:]

        cv2.imshow('Video', frame)

    cap.release()


Processing video #1...
Processing video #2...
Processing video #3...
Processing video #4...
Processing video #5...
Processing video #6...
Processing video #7...
Processing video #8...
Processing video #9...
Processing video #10...
Processing video #11...
Processing video #12...
Processing video #13...
Processing video #14...
Processing video #15...
Processing video #16...
Processing video #17...
Processing video #18...
Processing video #19...
Processing video #20...


## Dataset augmentation

In [9]:
def data_augmentation(image):
    # Esempio di augmentation: applica una rotazione casuale
    angle = np.random.randint(-10, 10)  # Angolo casuale tra -10 e 10 gradi
    M = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
    augmented_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    return augmented_image


In [10]:
for idx_video, video_path in enumerate(video_paths):
    cap = cv2.VideoCapture(video_path)
    print(f"Processing video #{idx_video+1}...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Applica l'augmentation
        frame_rgb = data_augmentation(frame_rgb)

        results = holistic.process(frame_rgb)

        if results.pose_landmarks:
            #right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            #right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
            right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]
            
            # Estrai le coordinate x, y, e z e normalizzale
            #shoulder_coords = [right_shoulder.x, right_shoulder.y, right_shoulder.z]
            #elbow_coords = [right_elbow.x, right_elbow.y, right_elbow.z]
            wrist_coords = [
                right_wrist.x, 
                right_wrist.y, 
                #right_wrist.z
                ]

            frame_buffer.append(
                # shoulder_coords
                #  + elbow_coords 
                wrist_coords
            )

            # Se il buffer contiene abbastanza frame, aggiungilo a X_training e y_training
            if len(frame_buffer) >= window_len:
                X_training.append(frame_buffer[:int(window_len/2)])
                y_training.append(frame_buffer[-1])
                frame_buffer = frame_buffer[1:]

        cv2.imshow('Video', frame)

    cap.release()

Processing video #1...
Processing video #2...
Processing video #3...
Processing video #4...
Processing video #5...
Processing video #6...
Processing video #7...
Processing video #8...
Processing video #9...
Processing video #10...
Processing video #11...
Processing video #12...
Processing video #13...
Processing video #14...
Processing video #15...
Processing video #16...
Processing video #17...
Processing video #18...
Processing video #19...
Processing video #20...


In [11]:
X_training = np.array(X_training)
y_training = np.array(y_training)
X_training.shape, y_training.shape

((4789, 5, 2), (4789, 2))

In [12]:
# Define the model
model = Sequential()

# Add an LSTM layer with input shape (.., ...)
model.add(LSTM(64, input_shape=(X_training.shape[1], X_training.shape[2]), activation='relu', return_sequences=True))

# Add another LSTM layer with return_sequences=False for the final prediction
model.add(LSTM(64, activation='relu', return_sequences=False))

# Add a dense output layer with 9 units (assuming you want to predict 9 values)
model.add(Dense(2))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 64)             17152     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 50306 (196.51 KB)
Trainable params: 50306 (196.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
batch_size = 32
epochs = 100

# Train the model
history = model.fit(X_training, y_training, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
120/120 [==============================] - 2s 5ms/step - loss: 0.0662 - accuracy: 0.8507 - val_loss: 0.0116 - val_accuracy: 0.8883
Epoch 2/100
120/120 [==============================] - 0s 3ms/step - loss: 0.0097 - accuracy: 0.8588 - val_loss: 0.0106 - val_accuracy: 0.8883
Epoch 3/100
120/120 [==============================] - 0s 3ms/step - loss: 0.0091 - accuracy: 0.8598 - val_loss: 0.0099 - val_accuracy: 0.8883
Epoch 4/100
120/120 [==============================] - 0s 3ms/step - loss: 0.0084 - accuracy: 0.8705 - val_loss: 0.0094 - val_accuracy: 0.8977
Epoch 5/100
120/120 [==============================] - 0s 3ms/step - loss: 0.0075 - accuracy: 0.8836 - val_loss: 0.0082 - val_accuracy: 0.8987
Epoch 6/100
120/120 [==============================] - 0s 3ms/step - loss: 0.0060 - accuracy: 0.8919 - val_loss: 0.0064 - val_accuracy: 0.9217
Epoch 7/100
120/120 [==============================] - 0s 3ms/step - loss: 0.0049 - accuracy: 0.9126 - val_loss: 0.0068 - val_accuracy: 0.9374

In [14]:
cap = cv2.VideoCapture(CAMERA_INDEX)
frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Converti il frame in scala di grigi
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Esegui il rilevamento delle pose con MediaPipe
    results = holistic.process(frame_rgb)

    # Estrai le coordinate della spalla destra, del gomito destro e della mano destra
    if results.pose_landmarks:
        #right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        #right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
        right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]

        # Le coordinate sono normalizzate, quindi convertile in coordinate pixel
        height, width, _ = frame.shape
        # shoulder_x, shoulder_y = int(right_shoulder.x * width), int(right_shoulder.y * height)
        # elbow_x, elbow_y = int(right_elbow.x * width), int(right_elbow.y * height)
        wrist_x, wrist_y = int(right_wrist.x * width), int(right_wrist.y * height)

        # Disegna i punti sul frame
        # cv2.circle(frame, (shoulder_x, shoulder_y), 5, (0, 0, 255), -1)
        # cv2.circle(frame, (elbow_x, elbow_y), 5, (0, 255, 0), -1)
        cv2.circle(frame, (wrist_x, wrist_y), 5, (255, 0, 0), -1)

        # Estrai le coordinate x, y, e z e normalizzale
        # shoulder_coords = [right_shoulder.x, right_shoulder.y, right_shoulder.z]
        # elbow_coords = [right_elbow.x, right_elbow.y, right_elbow.z]
        wrist_coords = [
            right_wrist.x,
            right_wrist.y,
            # right_wrist.z
        ]
        print(wrist_coords)

        # Logica per la predizione
        frame_buffer.append(
            # shoulder_coords
            # + elbow_coords
            wrist_coords
        )

        if len(frame_buffer) >= window_len/2:
            prediction_input = np.array(frame_buffer)[np.newaxis, :, :]
            prediction = model.predict(prediction_input)
            print(prediction)

            # Draw magenta circle for the predicted wrist position
            prediction_x = int(prediction[0][0] * width)
            prediction_y = int(prediction[0][1] * height)
            cv2.circle(frame, (prediction_x, prediction_y), 5, (255, 0, 255), -1)
            
            #Draw an arrow between the predicted and actual wrist position
            cv2.arrowedLine(frame, (wrist_x, wrist_y), (prediction_x, prediction_y), (255, 0, 255), 2)

            frame_buffer = frame_buffer[1:]

    # Visualizza il frame con le coordinate
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2023-10-11 19:41:36.398 python[35997:6139324] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


[0.37106162309646606, 1.444159746170044]
[0.3278725743293762, 1.7326263189315796]
[0.3195321261882782, 1.7544326782226562]
[0.3114495575428009, 1.7445000410079956]
[0.30116015672683716, 1.7856413125991821]
1/1 [==============================] - 0s 205ms/step
[[-0.4770394  1.3314931]]
[0.30716463923454285, 1.6437498331069946]
1/1 [==============================] - 0s 16ms/step
[[-0.20376833  1.2708335 ]]
[0.3132324814796448, 1.6409401893615723]
1/1 [==============================] - 0s 11ms/step
[[-0.10269609  1.2543894 ]]
[0.3140937387943268, 1.480137586593628]
1/1 [==============================] - 0s 22ms/step
[[0.11727878 1.1930085 ]]
[0.2761387228965759, 1.5831379890441895]
1/1 [==============================] - 0s 11ms/step
[[0.09215792 1.2284651 ]]
[0.09960293769836426, 1.4315462112426758]
1/1 [==============================] - 0s 11ms/step
[[-0.23712797  1.1953995 ]]
[0.0994243174791336, 1.3875079154968262]
1/1 [==============================] - 0s 10ms/step
[[-0.62110335  1.169